In [1]:
import sys
# sys.path.insert(0,'/eos/home-a/antoniov/SWAN_projects/env/uproot4/lib/python3.7/site-packages')
# sys.path.insert(0,'/eos/home-a/antoniov/SWAN_projects/env/uproot/lib/python3.8/site-packages')
sys.path.insert(0,'/eos/home-a/antoniov/SWAN_projects/env/uproot-py39/lib/python3.9/site-packages')
print ( sys.path )

import numpy as np
import pandas as pd
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import mplhep
from matplotlib.colors import LogNorm
from joblib import dump, load
# plt.style.use(mplhep.style.CMS)
import matplotlib.font_manager
mpl.rcParams.update(
    {'font.size': 18,
     'font.family': 'sans-serif',
     'legend.fontsize': 14,
     'axes.labelsize': 18,
     'xtick.labelsize': 14,
     'ytick.labelsize': 14
    }
    )

['/eos/home-a/antoniov/SWAN_projects/env/uproot-py39/lib/python3.9/site-packages', '/cvmfs/sft.cern.ch/lcg/releases/condor/8.9.11-f6d1a/x86_64-centos7-gcc11-opt/lib/python3', '/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/itk', '/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/python', '/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib', '', '/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages', '/srv/dask/lib', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.6-b0f98/x86_64-centos7-gcc8-opt/lib/python39.zip', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.6-b0f98/x86_64-centos7-gcc8-opt/lib/python3.9', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.6-b0f98/x86_64-centos7-gcc8-opt/lib/python3.9/lib-dynload', '/eos/home-a/antoniov/SWAN_projects/env/uproot-py39/lib/python3.9/site-packages', '/cvmfs/sft.cern.ch/lcg/views/LCG_103swan/x86_64-centos7-gcc11-opt/lib/python3.9/sit

In [2]:
from processing import lumi_periods_2017, lumi_periods_2018

lepton_type = 'muon'
# lepton_type = 'electron'

# data_sample = '2017'
data_sample = '2018'

save_figures = False

{'muon': {'2017B': 4.799881474, '2017C1': 5.785813941, '2017C2': 3.786684323, '2017D': 4.247682053, '2017E': 9.312832062, '2017F1': 1.738905587, '2017F2': 8.125575961, '2017F3': 3.674404546}, 'electron': {'2017B': 4.594096155565198, '2017C1': 5.521298099245362, '2017C2': 3.6135646891210857, '2017D': 4.0518639103567, '2017E': 8.906848461089172, '2017F1': 1.6584516422062712, '2017F2': 7.749629938012412, '2017F3': 3.504400870870218}}
Luminosity 2017 muon: 41.471779946999995
Luminosity 2017 electron: 39.60015376646642
{'muon': {'2018A': 14.025827145867588, '2018B1': 6.620869367493728, '2018B2': 0.430376623828928, '2018C': 6.891684998276784, '2018D1': 20.939902986506983, '2018D2': 10.856932131702669}, 'electron': {'2018A': 13.088399561509418, '2018B1': 6.629521091507798, '2018B2': 0.430939012174748, '2018C': 6.8915954055654725, '2018D1': 20.94983928140255, '2018D2': 10.862083907209522}}
Luminosity 2018 muon: 59.76559325367668
Luminosity 2018 electron: 58.85237825936951


### Event selection

In [3]:
def select_events( df_events, lepton_type, max_num_tracks = -1, label_jet=None, label_muon=None, label_electron=None ):

    max_num_tracks_ = max_num_tracks
    
#     label_jet_ = "_nom"
#     label_muon_ = "_nom"
#     label_WLep_ = "_nom"
#     label_WW_ = "_nom"
    label_jet_ = "_nom"
    if label_jet is not None: label_jet_ = label_jet
        
    label_muon_ = "_nom"
    if label_muon is not None: label_muon_ = label_muon
        
    label_electron_ = "_nom"
    if label_electron is not None: label_electron_ = label_electron
        
    label_WLep_ = "_nom"
    if label_muon is not None:
        label_WLep_ = label_muon
    elif label_electron is not None:
        label_WLep_ = label_electron
        
    label_WW_ = "_nom"
    if label_jet is not None:
        label_WW_ = label_jet
    elif label_muon is not None:
        label_WW_ = label_muon
    elif label_electron is not None:
        label_WW_ = label_electron
        
    label_xiCMS_ = "_nom"
    if label_jet is not None:
        label_xiCMS_ = label_jet
    elif label_muon is not None:
        label_xiCMS_ = label_muon
    elif label_electron is not None:
        label_xiCMS_ = label_electron
    
    lepton_var_phi_ = ""
    lepton_var_eta_ = ""
    if lepton_type == 'muon':
        lepton_var_phi_ = "muon0_phi"
        lepton_var_eta_ = "muon0_eta"
    elif lepton_type == 'electron':
        lepton_var_phi_ = "electron0_phi"
        lepton_var_eta_ = "electron0_eta"
        
    deltaPhi_jet_lep_ = ( df_events.loc[ :, "jet0_phi"] - df_events.loc[ :, lepton_var_phi_ ] )
    msk_dphi_ = ( deltaPhi_jet_lep_ < -np.pi )
    deltaPhi_jet_lep_.loc[ msk_dphi_ ] = deltaPhi_jet_lep_.loc[ msk_dphi_ ] + 2*np.pi
    msk_dphi_ = ( deltaPhi_jet_lep_ >= np.pi )
    deltaPhi_jet_lep_.loc[ msk_dphi_ ] = deltaPhi_jet_lep_.loc[ msk_dphi_ ] - 2*np.pi
    
    deltaEta_jet_lep_ = ( df_events.loc[ :, "jet0_eta"] - df_events.loc[ :, lepton_var_eta_ ] )
    deltaR_jet_lep_ = np.sqrt( ( deltaPhi_jet_lep_ ) ** 2 + ( deltaEta_jet_lep_ ) ** 2 )
    
    deltaPhi_jet_met_ = ( df_events.loc[ :, "jet0_phi"] - df_events.loc[ :, "met_phi"] )
    msk_dphi_ = ( deltaPhi_jet_met_ < -np.pi )
    deltaPhi_jet_met_.loc[ msk_dphi_ ] = deltaPhi_jet_met_.loc[ msk_dphi_ ] + 2*np.pi
    msk_dphi_ = ( deltaPhi_jet_met_ >= np.pi )
    deltaPhi_jet_met_.loc[ msk_dphi_ ] = deltaPhi_jet_met_.loc[ msk_dphi_ ] - 2*np.pi
    
    deltaPhi_jet_Wlep_ = ( df_events.loc[ :, "jet0_phi"] - df_events.loc[ :, "WLeptonicPhi"] )
    msk_dphi_ = ( deltaPhi_jet_Wlep_ < -np.pi )
    deltaPhi_jet_Wlep_.loc[ msk_dphi_ ] = deltaPhi_jet_Wlep_.loc[ msk_dphi_ ] + 2*np.pi
    msk_dphi_ = ( deltaPhi_jet_Wlep_ >= np.pi )
    deltaPhi_jet_Wlep_.loc[ msk_dphi_ ] = deltaPhi_jet_Wlep_.loc[ msk_dphi_ ] - 2*np.pi

    jet0_tau21_ = ( df_events.loc[ :, "jet0_tau2"] / df_events.loc[ :, "jet0_tau1"] )

    msk_All_ = np.full( df_events.shape[0], True, dtype=bool )
    
    msk_lep_ = None
    if lepton_type == 'muon':
        msk_lep_ = ( ( df_events.loc[ :, "muon0_pt" + label_muon_ ] >= 50.0 ) &
                     ( np.abs( df_events.loc[ :, "muon0_eta"] ) <= 2.4 ) & 
                     ( df_events.loc[ :, "muon0_iso"] <= 0.10 ) )
    elif lepton_type == 'electron':
        msk_lep_ = ( ( df_events.loc[ :, "electron0_pt" + label_electron_ ] >= 50.0 ) &
                     ( np.abs( df_events.loc[ :, "electron0_eta"] ) <= 2.4 ) )
        
#     msk_jet_ = ( ( df_events.loc[ :, "jet0_pt" + label_jet_] >= 200.0 ) &
#                  ( np.abs( df_events.loc[ :, "jet0_eta"] ) <= 2.4 ) )
    msk_jet_ = ( ( df_events.loc[ :, "jet0_pt" + label_jet_] >= 220.0 ) &
                 ( np.abs( df_events.loc[ :, "jet0_eta"] ) <= 2.4 ) )
    
    msk_jet_lep_ = ( deltaR_jet_lep_ >= np.pi/2 )
    
    msk_met_ = ( df_events.loc[ :, "met"] >= 40.0 )
    
    msk_jet_met_ = ( np.abs( deltaPhi_jet_met_ ) >= 2.0 )
    
    msk_pruned_mass_ = ( ( df_events.loc[ :, "jet0_corrmass" + label_jet_] >= 50.0 ) &
                         ( df_events.loc[ :, "jet0_corrmass" + label_jet_] <= 110.0 ) )
    
    msk_tau21_       = ( jet0_tau21_ <= 0.60 )
    
    msk_bjet_veto_   = ( df_events.loc[ :, "num_bjets_ak4"] == 0 )
    
#     msk_xiCMS_ = msk_All_.copy()
    msk_xiCMS_       = ( ( df_events.loc[ :, "xiCMS_45" + label_xiCMS_ ] <= 0.20 ) & ( df_events.loc[ :, "xiCMS_56" + label_xiCMS_ ] <= 0.20 ) )
    
    msk_Wlep_ = ( df_events.loc[ :, "WLeptonicPt" + label_WLep_ ] >= 200.0 )
    
    msk_jet_Wlep_ = ( np.abs( deltaPhi_jet_Wlep_ ) >= 2.5 )
    
#     msk_MWW_ = ( df_events.loc[ :, "recoMWW"] >= 500.0 )
    msk_MWW_ = ( df_events.loc[ :, "MWW" + label_WW_ ] >= 500.0 )

#     msk_extra_tracks_ = ( df_events.loc[ :, "pfcand_nextracks" ] <= 999 )    
    msk_extra_tracks_ = None
    if max_num_tracks_ >= 0:
        msk_extra_tracks_ = ( df_events.loc[ :, "pfcand_nextracks" ] <= max_num_tracks_ )
    else:
        msk_extra_tracks_ = np.full( df_events.shape[0], True, dtype=bool )
    
    masks_ = [ msk_All_, msk_lep_, msk_jet_, msk_jet_lep_, msk_met_, msk_jet_met_,
               msk_pruned_mass_, msk_tau21_, msk_bjet_veto_, msk_xiCMS_,
               msk_Wlep_, msk_jet_Wlep_, msk_MWW_, msk_extra_tracks_ ]
    
    lepton_sel_name_ = ""
    jet_lepton_sel_name_ = ""
    if lepton_type == 'muon':
        lepton_sel_name_ = "Muon"
        jet_lepton_sel_name_ = "Jet_Muon"
    elif lepton_type == 'electron':
        lepton_sel_name_ = "Electron"
        jet_lepton_sel_name_ = "Jet_Electron"
        
    selection_keys_ = [ "All", lepton_sel_name_, "Jet", jet_lepton_sel_name_, "MET", "Jet_MET",
                        "PrunedMass", "Tau21", "BJet_Veto", "Xi_CMS",
                        "WLeptonic", "Jet_WLeptonic", "MWW", "ExtraTracks" ]
    
    selections_ = {}
    msk_ = masks_[ 0 ].copy()
    for idx_, key_ in enumerate( selection_keys_ ):
        msk_ = msk_ & masks_[ idx_ ]
        selections_[ key_ ] = np.sum( msk_ )
        
    df_events = df_events.loc[ msk_ ]
    
    return df_events, selection_keys_, selections_

### Data (non-proton)

In [4]:
labels_data = []
fileNames_data = {}
if data_sample == '2017':
    if lepton_type == 'muon':
        labels_data = [ ]
        fileNames_data = {
            
        }
    elif lepton_type == 'electron':
        labels_data = [ ]
        fileNames_data = {
            
        }
elif data_sample == '2018':
    if lepton_type == 'muon':
        labels_data = [
            "data-2018-muon-2018A",
            "data-2018-muon-2018B",
            "data-2018-muon-2018C",
            "data-2018-muon-2018D"
        ]
        fileNames_data = {
            "data-2018-muon-2018A": "data-store-data-2018-muon-events-2018A.h5",
            "data-2018-muon-2018B": "data-store-data-2018-muon-events-2018B.h5",
            "data-2018-muon-2018C": "data-store-data-2018-muon-events-2018C.h5",
            "data-2018-muon-2018D": "data-store-data-2018-muon-events-2018D.h5"
        }
    elif lepton_type == 'electron':
        labels_data = [
            "data-2018-electron-2018A",
            "data-2018-electron-2018B",
            "data-2018-electron-2018C",
            "data-2018-electron-2018D"
        ]
        fileNames_data = {
            "data-2018-electron-2018A": "data-store-data-2018-electron-events-2018A.h5",
            "data-2018-electron-2018B": "data-store-data-2018-electron-events-2018B.h5",
            "data-2018-electron-2018C": "data-store-data-2018-electron-events-2018C.h5",
            "data-2018-electron-2018D": "data-store-data-2018-electron-events-2018D.h5"
        }

base_path_ = 'output'
for label_ in fileNames_data:
    fileNames_data[ label_ ] = "{}/{}".format( base_path_, fileNames_data[ label_ ] )
print ( fileNames_data )

df_counts_data = None
df_events_data = None

df_counts_data_list_ = []
df_events_data_list_ = []
for label__ in labels_data:
    print ( label__ )
    file_path_ = fileNames_data[ label__ ]
    print ( file_path_ )
    with pd.HDFStore( file_path_, 'r' ) as store_:
        print ( list( store_ ) )
        
        df_counts_data__ = store_[ "counts" ]
        df_events_data__ = store_[ "events" ]
        df_counts_data_list_.append( df_counts_data__ )
        df_events_data_list_.append( df_events_data__ )

df_counts_data = df_counts_data_list_[0]
for idx_ in range( 1, len( df_counts_data_list_ ) ):
    df_counts_data = df_counts_data.add( df_counts_data_list_[ idx_ ] )

df_events_data = pd.concat( df_events_data_list_ )

{'data-2018-muon-2018A': 'output/data-store-data-2018-muon-events-2018A.h5', 'data-2018-muon-2018B': 'output/data-store-data-2018-muon-events-2018B.h5', 'data-2018-muon-2018C': 'output/data-store-data-2018-muon-events-2018C.h5', 'data-2018-muon-2018D': 'output/data-store-data-2018-muon-events-2018D.h5'}
data-2018-muon-2018A
output/data-store-data-2018-muon-events-2018A.h5
['/counts', '/events']
data-2018-muon-2018B
output/data-store-data-2018-muon-events-2018B.h5
['/counts', '/events']
data-2018-muon-2018C
output/data-store-data-2018-muon-events-2018C.h5
['/counts', '/events']
data-2018-muon-2018D
output/data-store-data-2018-muon-events-2018D.h5
['/counts', '/events']


In [5]:
df_counts_data

All           2168978
Jet           2168978
Muon          2101524
check_none    2101524
dtype: int64

In [6]:
df_events_data

,run,lumiblock,event,crossingAngle,betaStar,instLumi,jet0_pt,jet0_eta,jet0_phi,jet0_energy,...,WLeptonicPy_nom,WLeptonicPz_nom,WLeptonicE_nom,WLeptonicM_nom,WW_energy_nom,WW_pz_nom,MWW_nom,YWW_nom,xiCMS_45_nom,xiCMS_56_nom
0,315488,496,475180672,150.0,0.3,10.037100,324.418854,-1.745987,-2.374501,962.026794,...,266.945923,-255.383423,403.616882,80.380127,1365.643677,-1156.794678,718.714417,-1.245685,0.015135,0.193103
1,315488,496,474492736,150.0,0.3,10.037100,254.027451,-1.322386,0.040865,515.584961,...,76.241447,232.864914,318.975616,80.380272,834.560547,-209.887894,800.355713,-0.257008,0.046865,0.079155
2,315488,496,474879328,150.0,0.3,10.037100,277.950073,-1.939334,0.408438,991.736572,...,-82.667419,-114.833130,254.945709,88.454399,1246.682251,-1061.297363,650.545715,-1.260847,0.012634,0.175911
3,315488,496,474548640,150.0,0.3,10.037100,430.706177,-1.419859,3.136551,958.454407,...,43.376720,131.947021,236.058884,80.380089,1194.513306,-706.807861,926.748779,-0.680296,0.035232,0.143972
4,315488,496,474906016,150.0,0.3,10.037100,234.374908,1.846084,2.145988,764.282471,...,-80.876724,457.071960,504.337677,80.379959,1268.620117,1180.951904,445.618256,1.665055,0.187671,0.005986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113877,325170,638,1175818880,133.0,0.3,9.299241,240.912888,0.785956,0.338386,323.060333,...,-70.913414,475.830414,554.740601,179.506393,877.800903,685.282349,548.215698,1.047112,0.117647,0.012219
113878,325170,646,1189408128,133.0,0.3,9.256189,244.522369,0.625718,-0.854012,307.663757,...,-34.330780,136.071136,194.756897,80.380089,502.420654,299.254395,335.163116,0.686339,0.059279,0.013240
113879,325170,646,1189031680,133.0,0.3,9.256189,280.787811,0.224587,0.903871,302.975403,...,-217.282745,28.342747,239.183044,80.380081,542.158447,91.935547,520.318115,0.171227,0.046547,0.032403
113880,325170,646,1189169920,133.0,0.3,9.256189,281.083374,1.122923,-0.892913,483.484070,...,106.177147,134.274094,203.753601,80.380150,687.237671,520.552673,422.513092,0.990221,0.091193,0.011108


### Data (protons)

In [7]:
labels_data = []
fileNames_data = {}
if data_sample == '2017':
    if lepton_type == 'muon':
        labels_data = [ "data-single-arm" ]
        fileNames_data = {
            "data-single-arm": "data-store-data-single-arm.h5"
        }
    elif lepton_type == 'electron':
        labels_data = [ "data-single-arm-electron" ]
        fileNames_data = {
            "data-single-arm-electron": "data-store-data-single-arm-electron.h5"
        }
elif data_sample == '2018':
    if lepton_type == 'muon':
        labels_data = [
            "data-2018-muon-2018A",
            "data-2018-muon-2018B",
            "data-2018-muon-2018C",
            "data-2018-muon-2018D"
        ]
        fileNames_data = {
            "data-2018-muon-2018A": "data-store-data-2018-muon-2018A.h5",
            "data-2018-muon-2018B": "data-store-data-2018-muon-2018B.h5",
            "data-2018-muon-2018C": "data-store-data-2018-muon-2018C.h5",
            "data-2018-muon-2018D": "data-store-data-2018-muon-2018D.h5"
        }
    elif lepton_type == 'electron':
        labels_data = [
            "data-2018-electron-2018A",
            "data-2018-electron-2018B",
            "data-2018-electron-2018C",
            "data-2018-electron-2018D"
        ]
        fileNames_data = {
            "data-2018-electron-2018A": "data-store-data-2018-electron-2018A.h5",
            "data-2018-electron-2018B": "data-store-data-2018-electron-2018B.h5",
            "data-2018-electron-2018C": "data-store-data-2018-electron-2018C.h5",
            "data-2018-electron-2018D": "data-store-data-2018-electron-2018D.h5"
        }

base_path_ = 'output'
# file_path_ = "{}/data-store-{}.h5".format( base_path_, label_data )
for label_ in fileNames_data:
    fileNames_data[ label_ ] = "{}/{}".format( base_path_, fileNames_data[ label_ ] )
print ( fileNames_data )

# df_counts_data = None
# # df_protons_multiRP_data_index = None
# df_protons_multiRP_data_events = None
# with pd.HDFStore( file_path_, 'r' ) as store_:
#     print ( list( store_ ) )
#     df_counts_data = store_[ "counts" ]
# #     df_protons_multiRP_data_index = store_[ "protons_multiRP" ]
#     df_protons_multiRP_data_events = store_[ "events_multiRP" ]

df_protons_multiRP_counts_data = None
# df_protons_multiRP_data_index = None
df_protons_multiRP_data_events = None

df_protons_multiRP_counts_data_list_ = []
# df_protons_multiRP_data_index_list_ = []
df_protons_multiRP_data_events_list_ = []
for label_ in labels_data:
    print ( label_ )
    file_path_ = fileNames_data[ label_ ]
    print ( file_path_ )
    with pd.HDFStore( file_path_, 'r' ) as store_:
        print ( list( store_ ) )
        
        df_counts_data__ = store_[ "counts" ]
#         df_protons_multiRP_data_index__  = store_[ "protons_multiRP" ]
        df_protons_multiRP_data_events__ = store_[ "events_multiRP" ]
        df_protons_multiRP_counts_data_list_.append( df_counts_data__ )
#         df_protons_multiRP_data_index_list_.append( df_protons_multiRP_data_index__ )
        df_protons_multiRP_data_events_list_.append( df_protons_multiRP_data_events__ )

df_protons_multiRP_counts_data = df_protons_multiRP_counts_data_list_[0]
for idx_ in range( 1, len( df_protons_multiRP_counts_data_list_ ) ):
    df_protons_multiRP_counts_data = df_protons_multiRP_counts_data.add( df_protons_multiRP_counts_data_list_[ idx_ ] )

# df_protons_multiRP_data_index = pd.concat( df_protons_multiRP_data_index_list_ )
df_protons_multiRP_data_events = pd.concat( df_protons_multiRP_data_events_list_ )

{'data-2018-muon-2018A': 'output/data-store-data-2018-muon-2018A.h5', 'data-2018-muon-2018B': 'output/data-store-data-2018-muon-2018B.h5', 'data-2018-muon-2018C': 'output/data-store-data-2018-muon-2018C.h5', 'data-2018-muon-2018D': 'output/data-store-data-2018-muon-2018D.h5'}
data-2018-muon-2018A
output/data-store-data-2018-muon-2018A.h5
['/counts', '/events_multiRP', '/protons_multiRP']
data-2018-muon-2018B
output/data-store-data-2018-muon-2018B.h5
['/counts', '/events_multiRP', '/protons_multiRP']
data-2018-muon-2018C
output/data-store-data-2018-muon-2018C.h5
['/counts', '/events_multiRP', '/protons_multiRP']
data-2018-muon-2018D
output/data-store-data-2018-muon-2018D.h5
['/counts', '/events_multiRP', '/protons_multiRP']


In [8]:
df_protons_multiRP_counts_data

All                  2168978
Jet                  2168978
Muon                 2101524
Proton_check_none    2101524
Proton               1436091
dtype: int64

In [9]:
df_protons_multiRP_data_events

crossingAngle  betaStar   instLumi  \
run    lumiblock event      slice                                       
316199 738       995631040  0              143.0       0.3  13.800958   
                 995580288  0              143.0       0.3  13.800958   
                 995377984  0              143.0       0.3  13.800958   
                 995499584  0              143.0       0.3  13.800958   
       739       997434176  0              143.0       0.3  13.797550   
...                                          ...       ...        ...   
325170 612       1131288576 0              133.0       0.3   9.424077   
       636       1171731200 0              133.0       0.3   9.308405   
                 1172657792 0              133.0       0.3   9.308405   
                 1172100864 0              133.0       0.3   9.308405   
       637       1174399232 0              133.0       0.3   9.307345   

                                      jet0_pt  jet0_eta  jet0_phi  \
run    lumiblock event      slice                                   
316199 738       995631040  0      255.131989  0.222181  0.465844   
                 995580288  0      271.237091  1.201405 -0.672793   
                 995377984  0      229.029266  0.813785  2.755530   
                 995499584  0      255.765457 -0.307157  0.577788   
       739       997434176  0      260.124023 -1.864394 -2.418813   
...                                       ...       ...       ...   
325170 612       1131288576 0      246.729645  1.276864 -0.892836   
       636       1171731200 0      228.047119  0.597068  2.040150   
                 1172657792 0      379.605713  0.926268  1.182455   
                 1172100864 0      251.445709 -0.306289  2.033631   
       637       1174399232 0      387.555847 -1.455234  2.785599   

                                   jet0_energy   jet0_mass  jet0_corrmass  \
run    lumiblock event      slice                                           
316199 738       995631040  0       267.062378   11.652953      12.641805   
                 995580288  0       499.218048   58.163593      65.961021   
                 995377984  0       314.838440   17.131739      18.583464   
                 995499584  0       282.566986   68.186745      73.808731   
       739       997434176  0       862.344360   16.077612      18.077419   
...                                        ...         ...            ...   
325170 612       1131288576 0       479.071381    8.522972       9.892778   
       636       1171731200 0       274.537445   12.774544      13.872478   
                 1172657792 0       563.302612   45.636650      50.065033   
                 1172100864 0       275.799347   64.325394      70.173836   
       637       1174399232 0       886.254150  103.373039     117.672409   

                                   jet0_tau1  ...           MX        YX  \
run    lumiblock event      slice             ...                          
316199 738       995631040  0       0.114730  ...  1081.969849 -0.666323   
                 995580288  0       0.327002  ...  1574.876953 -0.395413   
                 995377984  0       0.153312  ...   856.215759 -0.402162   
                 995499584  0       0.384951  ...   916.753662  0.275115   
       739       997434176  0       0.175540  ...  1315.981689 -0.325852   
...                                      ...  ...          ...       ...   
325170 612       1131288576 0       0.102824  ...   893.898987  0.588127   
       636       1171731200 0       0.148284  ...   937.104431 -0.351514   
                 1172657792 0       0.229042  ...  1345.170898 -0.256506   
                 1172100864 0       0.306301  ...  1131.908936 -0.433039   
       637       1174399232 0       0.307969  ...  1221.715698 -0.098217   

                                   diffMWW_MX  ratioMWW_MX  \
run    lumiblock event      slice                            
316199 738       995631040  0     -641.624390     0.406985   
          

### Select events (non-proton)
#### Without track multiplicity selection

In [10]:
df_events_data__, selection_keys_, selections_ = select_events( df_events_data, lepton_type, max_num_tracks = -1 )
df_events_data = df_events_data__
print ( df_events_data.shape[0] )
print ( selection_keys_ )
print ( selections_ )

81307
['All', 'Muon', 'Jet', 'Jet_Muon', 'MET', 'Jet_MET', 'PrunedMass', 'Tau21', 'BJet_Veto', 'Xi_CMS', 'WLeptonic', 'Jet_WLeptonic', 'MWW', 'ExtraTracks']
{'All': 2101524, 'Muon': 2098271, 'Jet': 1502978, 'Jet_Muon': 1484114, 'MET': 1151529, 'Jet_MET': 1032122, 'PrunedMass': 284410, 'Tau21': 190848, 'BJet_Veto': 152257, 'Xi_CMS': 134927, 'WLeptonic': 88552, 'Jet_WLeptonic': 86362, 'MWW': 81307, 'ExtraTracks': 81307}


In [11]:
df_events_data

,run,lumiblock,event,crossingAngle,betaStar,instLumi,jet0_pt,jet0_eta,jet0_phi,jet0_energy,...,WLeptonicPy_nom,WLeptonicPz_nom,WLeptonicE_nom,WLeptonicM_nom,WW_energy_nom,WW_pz_nom,MWW_nom,YWW_nom,xiCMS_45_nom,xiCMS_56_nom
19,315488,498,476996864,150.0,0.3,10.024202,291.053375,-1.152888,-2.762830,521.336304,...,101.748436,0.133389,289.671448,80.380081,811.007751,-414.851166,696.755127,-0.564794,0.028486,0.092310
59,315488,636,593238336,148.0,0.3,9.177520,279.357178,1.253695,2.997357,541.728516,...,-36.185795,-531.239014,592.105713,80.379959,1133.834229,-81.778839,1130.468872,-0.072251,0.079542,0.092123
105,315645,113,120282632,146.0,0.3,13.027858,307.076508,-0.504233,1.300769,364.346375,...,-216.110062,-176.631790,308.226624,80.380081,672.572998,-338.115051,575.506165,-0.552938,0.023569,0.075586
174,316199,739,997501056,143.0,0.3,13.797550,239.512299,-0.711949,3.141166,315.100525,...,7.491745,2.170815,222.543488,80.380157,537.644043,-183.124481,504.416473,-0.354777,0.025171,0.053344
200,316199,741,1001800576,143.0,0.3,13.781249,302.460541,0.704580,-3.118840,398.335205,...,38.795143,-35.391033,273.984924,80.380135,672.320129,195.791672,640.754639,0.299897,0.064493,0.034372
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113808,325170,601,1112116352,133.0,0.3,9.480412,278.396271,1.518540,0.306455,674.143005,...,-118.790565,345.692200,450.699188,80.380104,1124.842163,950.720032,599.987549,1.239115,0.158477,0.012212
113811,325170,609,1125037312,133.0,0.3,9.441976,242.900787,1.097676,-2.862669,409.564758,...,84.010933,528.248840,599.454102,80.379959,1009.018860,851.737549,540.137329,1.235351,0.142332,0.011295
113844,325170,635,1170101632,133.0,0.3,9.324137,444.657990,-0.047140,-1.622253,452.845642,...,437.571472,-10.604792,445.020538,80.380005,897.866211,-31.573639,896.968750,-0.035180,0.065483,0.070341
113860,325170,649,1194329984,132.0,0.3,9.260224,252.083542,-0.747955,2.854759,334.191833,...,-61.345764,135.174728,272.733429,90.522194,606.925293,-71.450882,601.790222,-0.118274,0.039367,0.050359


### Select events (protons)
#### Without track multiplicity selection

In [12]:
df_protons_multiRP_data_events__, selection_keys_, selections_ = select_events( df_protons_multiRP_data_events, lepton_type, max_num_tracks = -1 )
df_protons_multiRP_data_events = df_protons_multiRP_data_events__
print ( df_protons_multiRP_data_events.shape[0] )
print ( selection_keys_ )
print ( selections_ )

15194
['All', 'Muon', 'Jet', 'Jet_Muon', 'MET', 'Jet_MET', 'PrunedMass', 'Tau21', 'BJet_Veto', 'Xi_CMS', 'WLeptonic', 'Jet_WLeptonic', 'MWW', 'ExtraTracks']
{'All': 400491, 'Muon': 399843, 'Jet': 286221, 'Jet_Muon': 282658, 'MET': 219545, 'Jet_MET': 196539, 'PrunedMass': 54611, 'Tau21': 35575, 'BJet_Veto': 28561, 'Xi_CMS': 25312, 'WLeptonic': 16512, 'Jet_WLeptonic': 16134, 'MWW': 15194, 'ExtraTracks': 15194}


In [13]:
df_protons_multiRP_data_events

crossingAngle  betaStar   instLumi  \
run    lumiblock event      slice                                       
316199 739       997501056  0              143.0       0.3  13.797550   
       741       1000536064 0              143.0       0.3  13.781249   
       458       641690048  0              147.0       0.3  15.685783   
       478       667656512  0              147.0       0.3  15.536963   
       479       668202176  0              147.0       0.3  15.513849   
...                                          ...       ...        ...   
325170 591       1094472832 0              134.0       0.3   9.521908   
       561       1040876096 0              134.0       0.3   9.675070   
       565       1048768384 0              134.0       0.3   9.649692   
       567       1052482304 0              134.0       0.3   9.634359   
       574       1064557568 0              134.0       0.3   9.590843   

                                      jet0_pt  jet0_eta  jet0_phi  \
run    lumiblock event      slice                                   
316199 739       997501056  0      239.512299 -0.711949  3.141166   
       741       1000536064 0      276.746094  0.734669 -2.169110   
       458       641690048  0      316.206085 -1.076036 -0.586171   
       478       667656512  0      313.841187  1.163683 -2.941736   
       479       668202176  0      320.559021 -0.015873 -1.620912   
...                                       ...       ...       ...   
325170 591       1094472832 0      290.629211 -1.097450 -1.881312   
       561       1040876096 0      334.187347 -0.316409  1.129602   
       565       1048768384 0      263.620880 -1.415727 -2.448367   
       567       1052482304 0      309.762909 -0.456844 -2.794641   
       574       1064557568 0      226.876648  0.247692 -1.912746   

                                   jet0_energy  jet0_mass  jet0_corrmass  \
run    lumiblock event      slice                                          
316199 739       997501056  0       315.100525  66.187607      71.980003   
       741       1000536064 0       367.181458  73.411423      79.435020   
       458       641690048  0       526.585938  54.371677      60.295959   
       478       667656512  0       563.642029  88.575485     100.295494   
       479       668202176  0       335.721649  63.186100      68.737488   
...                                        ...        ...            ...   
325170 591       1094472832 0       496.245667  79.000069      88.709236   
       561       1040876096 0       370.368225  87.412888      95.330956   
       565       1048768384 0       584.100952  76.736046      88.183403   
       567       1052482304 0       350.336914  50.643635      55.437069   
       574       1064557568 0       253.780975  79.938332      87.225609   

                                   jet0_tau1  ...           MX        YX  \
run    lumiblock event      slice             ...                          
316199 739       997501056  0       0.367093  ...  1416.240967 -0.499026   
       741       1000536064 0       0.371637  ...  1503.715698 -0.214034   
       458       641690048  0       0.270401  ...  1208.016724 -0.139538   
       478       667656512  0       0.266785  ...  1638.921143 -0.246431   
       479       668202176  0       0.307667  ...   733.583313 -0.640865   
...                                      ...  ...          ...       ...   
325170 591       1094472832 0       0.356889  ...   748.573608 -0.010486   
       561       1040876096 0       0.261081  ...  1432.937988 -0.193653   
       565       1048768384 0       0.353419  ...  1526.097412 -0.049477   
       567       1052482304 0       0.182705  ...   806.620789 -0.239772   
       574       1064557568 0       0.469280  ...   965.018372 -0.441832   

                                   diffMWW_MX  ratioMWW_MX  \
run    lumiblock event      slice                            
316199 739       997501056  0     -911.765808     0.356207   
       741       100053

### Remove proton-tagged events from data event sample

In [14]:
index_vars_ = ['run', 'lumiblock', 'event' ]
df_events_data = df_events_data.set_index( index_vars_ )
df_protons_multiRP_data_events = df_protons_multiRP_data_events.reset_index().set_index( index_vars_ )

In [15]:
df_events_data.index

MultiIndex([(315488, 498,  476996864),
            (315488, 636,  593238336),
            (315645, 113,  120282632),
            (316199, 739,  997501056),
            (316199, 741, 1001800576),
            (316199, 741, 1000536064),
            (316199, 419,  590030080),
            (316199, 441,  619148096),
            (316199, 438,  615407616),
            (316199, 431,  606212352),
            ...
            (325170, 579, 1072557184),
            (325170, 579, 1072659712),
            (325170, 583, 1080110336),
            (325170, 574, 1064557568),
            (325170, 596, 1103380224),
            (325170, 601, 1112116352),
            (325170, 609, 1125037312),
            (325170, 635, 1170101632),
            (325170, 649, 1194329984),
            (325170, 638, 1175496320)],
           names=['run', 'lumiblock', 'event'], length=81307)

In [16]:
df_events_data = df_events_data.drop( index=df_protons_multiRP_data_events.index )

In [17]:
df_events_data

crossingAngle  betaStar   instLumi     jet0_pt  \
run    lumiblock event                                                        
315488 498       476996864           150.0       0.3  10.024202  291.053375   
       636       593238336           148.0       0.3   9.177520  279.357178   
315645 113       120282632           146.0       0.3  13.027858  307.076508   
316199 741       1001800576          143.0       0.3  13.781249  302.460541   
       419       590030080           148.0       0.3  16.124950  286.730804   
...                                    ...       ...        ...         ...   
325170 601       1112116352          133.0       0.3   9.480412  278.396271   
       609       1125037312          133.0       0.3   9.441976  242.900787   
       635       1170101632          133.0       0.3   9.324137  444.657990   
       649       1194329984          132.0       0.3   9.260224  252.083542   
       638       1175496320          133.0       0.3   9.299241  255.183746   

                             jet0_eta  jet0_phi  jet0_energy  jet0_mass  \
run    lumiblock event                                                    
315488 498       476996864  -1.152888 -2.762830   521.336304  96.102020   
       636       593238336   1.253695  2.997357   541.728516  86.257507   
315645 113       120282632  -0.504233  1.300769   364.346375  69.359520   
316199 741       1001800576  0.704580 -3.118840   398.335205  91.754852   
       419       590030080   0.229567  1.229858   311.618164  89.541756   
...                               ...       ...          ...        ...   
325170 601       1112116352  1.518540  0.306455   674.143005  72.003456   
       609       1125037312  1.097676 -2.862669   409.564758  45.786453   
       635       1170101632 -0.047140 -1.622253   452.845642  58.509716   
       649       1194329984 -0.747955  2.854759   334.191833  51.986931   
       638       1175496320 -0.733967  1.854041   339.455627  77.893600   

                             jet0_corrmass  jet0_tau1  ...  WLeptonicPy_nom  \
run    lumiblock event                                 ...                    
315488 498       476996864      108.822762   0.380043  ...       101.748436   
       636       593238336       98.918289   0.436552  ...       -36.185795   
315645 113       120282632       75.313492   0.311238  ...      -216.110062   
316199 741       1001800576      99.232704   0.422205  ...        38.795143   
       419       590030080       97.144554   0.345608  ...      -156.105911   
...                                    ...        ...  ...              ...   
325170 601       1112116352      81.723534   0.303951  ...      -118.790565   
       609       1125037312      51.556835   0.167413  ...        84.010933   
       635       1170101632      64.082352   0.179790  ...       437.571472   
       649       1194329984      56.944401   0.264462  ...       -61.345764   
       638       1175496320      85.321350   0.322466  ...      -228.373917   

                             WLeptonicPz_nom  WLeptonicE_nom  WLeptonicM_nom  \
run    lumiblock event                                                         
315488 498       476996864          0.133389      289.671448       80.380081   
       636       593238336       -531.239014      592.105713       80.379959   
315645 113       120282632       -176.631790      308.226624       80.380081   
316199 741       1001800576       -35.391033      273.984924       80.380135   
       419       590030080        128.338379      253.588348       87.737503   
...                                      ...             ...             ...   
325170 601       1112116352       345.692200      450.699188       80.380104   
       609       1125037312       528.248840      599.454102       80.379959   
       635       1170101632       -10.604792      445.020538       80.380005   
       649       1194329984       135.174728      272.733429       90.522194   
       638       1175496320      -328

### Track multiplicity selection

In [18]:
# df_events_data__, selection_keys_, selections_ = select_events( df_events_data, lepton_type, max_num_tracks = 4 )
# df_events_data_nTracks_4 = df_events_data__
# print ( df_events_data_nTracks_4.shape[0] )
# print ( selection_keys_ )
# print ( selections_ )

df_events_data_nTracks_4 = df_events_data.loc[ ( df_events_data.loc[ :, "pfcand_nextracks" ] <= 4 ) ]

In [19]:
df_events_data_nTracks_4

,,,crossingAngle,betaStar,instLumi,jet0_pt,jet0_eta,jet0_phi,jet0_energy,jet0_mass,jet0_corrmass,jet0_tau1,...,WLeptonicPy_nom,WLeptonicPz_nom,WLeptonicE_nom,WLeptonicM_nom,WW_energy_nom,WW_pz_nom,MWW_nom,YWW_nom,xiCMS_45_nom,xiCMS_56_nom
run,lumiblock,event,,,,,,,,,,,,,,,,,,,,,
315645,279,282676384,144.0,0.30,11.797978,280.120453,1.302534,1.614148,559.671082,56.200203,64.452576,0.255488,...,-242.063248,90.940887,271.007782,80.380081,830.678833,568.092896,604.431396,0.836386,0.106170,0.018771
316201,111,134589152,133.0,0.30,10.268744,239.372498,1.616366,1.192000,632.738831,68.184700,75.380463,0.385913,...,-223.399414,-29.848017,253.209137,80.380112,885.947998,548.971558,695.341919,0.724426,0.108881,0.024423
315557,192,212459168,148.0,0.30,12.756579,370.291992,1.319601,-3.104910,746.825073,53.237854,60.708885,0.168398,...,2.713531,250.171677,418.752319,80.380051,1165.577393,893.495850,747.115173,1.011955,0.157442,0.019981
316590,264,373348672,149.0,0.30,16.123880,226.785736,-1.968353,1.681514,834.249451,74.993683,83.877609,0.467809,...,-209.984818,-64.330551,237.165359,80.380089,1071.414795,-860.256287,638.313965,-1.106766,0.014652,0.146999
316766,56,32192888,160.0,0.30,18.657238,276.317383,-1.868553,2.954905,919.697083,47.445015,53.112541,0.281935,...,-30.451242,-75.865173,266.034790,80.379967,1185.731934,-949.659180,709.002319,-1.101133,0.016953,0.163054
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324293,794,1480298752,139.0,0.30,11.642282,452.942749,-0.027698,-1.580886,461.570648,47.929161,52.486988,0.108369,...,344.566162,-118.439125,386.733032,80.379875,848.303711,-130.986282,824.271057,-0.155655,0.053878,0.074030
324420,89,189042400,153.0,0.30,16.206888,266.013916,-1.324561,-1.600452,542.787109,70.911240,81.830147,0.288675,...,266.211273,-30.438568,280.436737,80.380066,823.223877,-495.246368,657.486450,-0.695641,0.023767,0.099959
324970,2110,3705011968,130.0,0.25,5.429813,297.747986,-0.549386,-1.241496,351.340179,54.874302,59.651989,0.217885,...,264.028381,18.406460,280.394318,80.380142,631.734497,-153.525986,610.466370,-0.247984,0.035302,0.058921


In [20]:
# df_events_data_nTracks_inverted = df_events_data.drop( index=df_events_data_nTracks_4.index )

df_events_data_nTracks_inverted = df_events_data.loc[ ( df_events_data.loc[ :, "pfcand_nextracks" ] > 4 ) ]

In [21]:
df_events_data_nTracks_inverted

crossingAngle  betaStar   instLumi     jet0_pt  \
run    lumiblock event                                                        
315488 498       476996864           150.0       0.3  10.024202  291.053375   
       636       593238336           148.0       0.3   9.177520  279.357178   
315645 113       120282632           146.0       0.3  13.027858  307.076508   
316199 741       1001800576          143.0       0.3  13.781249  302.460541   
       419       590030080           148.0       0.3  16.124950  286.730804   
...                                    ...       ...        ...         ...   
325170 601       1112116352          133.0       0.3   9.480412  278.396271   
       609       1125037312          133.0       0.3   9.441976  242.900787   
       635       1170101632          133.0       0.3   9.324137  444.657990   
       649       1194329984          132.0       0.3   9.260224  252.083542   
       638       1175496320          133.0       0.3   9.299241  255.183746   

                             jet0_eta  jet0_phi  jet0_energy  jet0_mass  \
run    lumiblock event                                                    
315488 498       476996864  -1.152888 -2.762830   521.336304  96.102020   
       636       593238336   1.253695  2.997357   541.728516  86.257507   
315645 113       120282632  -0.504233  1.300769   364.346375  69.359520   
316199 741       1001800576  0.704580 -3.118840   398.335205  91.754852   
       419       590030080   0.229567  1.229858   311.618164  89.541756   
...                               ...       ...          ...        ...   
325170 601       1112116352  1.518540  0.306455   674.143005  72.003456   
       609       1125037312  1.097676 -2.862669   409.564758  45.786453   
       635       1170101632 -0.047140 -1.622253   452.845642  58.509716   
       649       1194329984 -0.747955  2.854759   334.191833  51.986931   
       638       1175496320 -0.733967  1.854041   339.455627  77.893600   

                             jet0_corrmass  jet0_tau1  ...  WLeptonicPy_nom  \
run    lumiblock event                                 ...                    
315488 498       476996864      108.822762   0.380043  ...       101.748436   
       636       593238336       98.918289   0.436552  ...       -36.185795   
315645 113       120282632       75.313492   0.311238  ...      -216.110062   
316199 741       1001800576      99.232704   0.422205  ...        38.795143   
       419       590030080       97.144554   0.345608  ...      -156.105911   
...                                    ...        ...  ...              ...   
325170 601       1112116352      81.723534   0.303951  ...      -118.790565   
       609       1125037312      51.556835   0.167413  ...        84.010933   
       635       1170101632      64.082352   0.179790  ...       437.571472   
       649       1194329984      56.944401   0.264462  ...       -61.345764   
       638       1175496320      85.321350   0.322466  ...      -228.373917   

                             WLeptonicPz_nom  WLeptonicE_nom  WLeptonicM_nom  \
run    lumiblock event                                                         
315488 498       476996864          0.133389      289.671448       80.380081   
       636       593238336       -531.239014      592.105713       80.379959   
315645 113       120282632       -176.631790      308.226624       80.380081   
316199 741       1001800576       -35.391033      273.984924       80.380135   
       419       590030080        128.338379      253.588348       87.737503   
...                                      ...             ...             ...   
325170 601       1112116352       345.692200      450.699188       80.380104   
       609       1125037312       528.248840      599.454102       80.379959   
       635       1170101632       -10.604792      445.020538       80.380005   
       649       1194329984       135.174728      272.733429       90.522194   
       638       1175496320      -328

In [22]:
df_events_data_nTracks_4.shape[0] / df_events_data_nTracks_inverted.shape[0]

0.0031560579622183444